In [1]:
import pandas as pd
from functools import reduce

# 1. 합칠 파일 리스트 정의 (상림이가 가진 7개 파일 이름을 여기 다 적어줘!)
file_names = [
    './데이터/카드회원정보_WideFormat_50000_463.csv',
    './데이터/카드청구정보_WideFormat_50000_265.csv',
    './데이터/카드채널정보_WideFormat_50000_619.csv',
    './데이터/카드잔액정보_WideFormat_50000_487.csv',
    './데이터/카드신용정보_WideFormat_50000_241.csv',
    './데이터/카드승인정보_WideFormat_50000_2569.csv',
    './데이터/카드성과정보_WideFormat_50000_283.csv'
]

# 2. 데이터를 안전하게 불러오는 함수
def load_data(file_name):
    try:
        # 한글 깨짐 방지를 위해 utf-8-sig 먼저 시도!
        return pd.read_csv(file_name, encoding='utf-8-sig')
    except:
        # 안되면 cp949로 시도할게
        return pd.read_csv(file_name, encoding='cp949')

In [2]:
# 3. 모든 파일을 불러와서 리스트에 담기
df_list = [load_data(name) for name in file_names]

# 4. '발급회원번호'를 기준으로 순차적으로 병합 (Merge)
# how='inner'는 모든 파일에 다 존재하는 회원만 남기는 거고,
# how='left'는 첫 번째 파일(회원정보) 기준으로 다 붙이는 거야. 상림이 상황에 맞춰 선택해!
df_merged = reduce(lambda left, right: pd.merge(left, right, on='발급회원번호', how='inner'), df_list)

# 5. 결과 확인
print("병합 완료!")
print(f"최종 통합 데이터 크기: {df_merged.shape}")
display(df_merged.head())

/var/folders/3p/8hc9c44s38j902ytq253kt6m0000gn/T/ipykernel_7168/3274641965.py:19: DtypeWarning: Columns (1223,1224,1232,1233,1234,1235,1236,1741,1742,1743,1744,1745,1746) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_name, encoding='utf-8-sig')


병합 완료!
최종 통합 데이터 크기: (50000, 4921)


,발급회원번호,남녀구분코드_07,남녀구분코드_08,남녀구분코드_09,남녀구분코드_10,남녀구분코드_11,남녀구분코드_12,연령_07,연령_08,연령_09,...,혜택수혜율_R3M_09,혜택수혜율_R3M_10,혜택수혜율_R3M_11,혜택수혜율_R3M_12,혜택수혜율_B0M_07,혜택수혜율_B0M_08,혜택수혜율_B0M_09,혜택수혜율_B0M_10,혜택수혜율_B0M_11,혜택수혜율_B0M_12
0,SYN_1000048,1,1,1,1,1,1,30대,30대,30대,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,SYN_1000194,1,1,1,1,1,1,50대,50대,50대,...,0.698266,1.451076,2.193782,0.000000,0.000000,2.529455,0.000000,0.000000,0.000000,0.000000
2,SYN_1000201,2,2,2,2,2,2,40대,40대,40대,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,SYN_100022,1,1,1,1,1,1,30대,30대,30대,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,SYN_1000250,1,1,1,1,1,1,40대,40대,40대,...,3.428598,2.661275,2.820443,2.677751,3.425927,4.148356,3.359937,3.430627,4.051669,4.173795


In [3]:
# 파일 이름을 상림이가 원하는 대로 정해줘!
output_file_name = '50k_wideformat_all_50000_4921.csv'

# index=False: 앞에 붙는 의미 없는 숫자(0, 1, 2...)는 저장 안 함
# encoding='utf-8-sig': 엑셀에서 열었을 때 한글이 깨지는 걸 막아줘
df_merged.to_csv(output_file_name, index=False, encoding='utf-8-sig')

In [4]:
# 1. VIP ID 파일 읽기
with open('vip_발급회원번호_ids.txt', 'r') as f:
    # 한 줄씩 읽어서 공백 제거하고 리스트로 만들어
    vip_ids = [line.strip() for line in f.readlines()]

print(f"상림아, 총 {len(vip_ids)}명의 VIP 회원 번호를 확인했어!")

상림아, 총 9894명의 VIP 회원 번호를 확인했어!


In [5]:
# 2. 통합 데이터에서 VIP와 General 나누기
# isin()을 쓰면 "이 명단에 있니?"라고 물어볼 수 있어!
df_vip = df_merged[df_merged['발급회원번호'].isin(vip_ids)]

# ~ 기호를 붙이면 "명단에 없는 사람(General)"만 골라내!
df_general = df_merged[~df_merged['발급회원번호'].isin(vip_ids)]

print(f"VIP 고객 수: {len(df_vip)}명")
print(f"일반 고객 수: {len(df_general)}명")

VIP 고객 수: 9894명
일반 고객 수: 40106명


In [6]:
df_vip.to_csv('50K_VIP_WiderFormat.csv', index=False, encoding='utf-8-sig')
df_general.to_csv('50K_General_WiderFormat.csv', index=False, encoding='utf-8-sig')